# Figures 2(c), 3(a)

Requires the respective script to be run first. See ```grande_experiment_slurm.sh``` for details.

In [1]:
import os
os.chdir("../")

In [2]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import glob
import json
from src import utils
from IPython.display import clear_output
import matplotlib.ticker as mtick
import numpy as np

sns.set_theme(context='paper', style='ticks', font_scale=1)

In [3]:
name="kappas"
width_pt = 397
palette = sns.color_palette('husl', 5)

In [4]:
files = glob.glob("./outputs/experiments/{name}_*.json".format(name=name))

dicts = []
for fl_id, fl in enumerate(files):
    clear_output(wait=True)
    print('Reading file ' + str(fl_id+1)+'/'+str(len(files)))
    with open(fl,"r") as f:
        js = json.load(f)
        
        dict = {}
        dict['k'] = js['k']
        dict['reward'] = js['reward']
        dict['cf_reward'] = js['cf_reward']
        dict['horizon'] = js['horizon']
        dict['pid'] = js['pid']
        dict['anchor_runtime'] = js['anchor_runtime']
        dict['astar_runtime'] = js['astar_runtime']
        dict['ebf'] = js['ebf']
        dict['visited'] = js['visited']

        dict['states'] = js['states']
        dict['cf_states'] = js['cf_states']
        dicts.append(dict)

raw_df = pd.DataFrame(dicts)

Reading file 2599/2599


In [5]:
input_df = raw_df[raw_df['horizon']==12].copy()

In [7]:
utils.latexify() # Computer Modern, with TeX

fig_width, fig_height = utils.get_fig_dim(width_pt, fraction=0.6)
fig, ax = plt.subplots(nrows=1, ncols=1, figsize=(fig_width, fig_height))
ax2 = ax.twinx()

# Draw Axis 1
sns.pointplot(data=input_df, x="k", y="ebf", capsize=0.3, color=palette[0], ax=ax)
sns.pointplot(data=input_df, x="k", y="astar_runtime", capsize=0.3, color=palette[2], ax=ax2)
sns.despine(top=True, right=False)
ax.set_xlabel(r"\# of action changes, $k$")
ax.set_ylabel("EBF")
ax2.set_ylabel(r"$A^*$ average runtime (s)")
ax.set_ylim(ymin=0.7, ymax=3)
ax2.set_ylim(ymax=600)

ax.tick_params(axis='y', color=palette[0])
ax2.tick_params(axis='y', color=palette[2])
ax2.spines['left'].set_color(palette[0])
ax2.spines['right'].set_color(palette[2])

fig.tight_layout()
fig.savefig('figures/kappas_t.pdf', dpi=300)

In [8]:
# get scaler data
with open("./data/processed/feature_normalization.json", 'r') as f:
    scaling = json.load(f)
    sofa_min = scaling['min']['SOFA']
    sofa_max = scaling['max']['SOFA']

def compute_sofa(row):
    states = np.array(row['states'])
    cf_states = np.array(row['cf_states'])
    states[:,-1] = (states[:,-1] + 0.5) * (sofa_max - sofa_min) + sofa_min
    cf_states[:,-1] = (cf_states[:,-1] + 0.5) * (sofa_max - sofa_min) + sofa_min
    # truncate to sofa_min, sofa_max
    states[:,-1] = np.clip(states[:,-1], sofa_min, sofa_max)
    cf_states[:,-1] = np.clip(cf_states[:,-1], sofa_min, sofa_max)
    return np.sum(states[:,-1]), np.sum(cf_states[:,-1])

In [9]:
# compute the cumulative observed and counterfactual sofa
for i in input_df.index:
    input_df.loc[i, 'total_sofa'], input_df.loc[i, 'cf_total_sofa'] = compute_sofa(input_df.loc[i])

input_df['improvement'] = (input_df['total_sofa'] - input_df['cf_total_sofa'])/input_df['total_sofa'] * 100

In [10]:
utils.latexify() # Computer Modern, with TeX

fig_width, fig_height = utils.get_fig_dim(width_pt, fraction=0.6)
fig, ax = plt.subplots(nrows=1, ncols=1, figsize=(fig_width, fig_height))

# Draw Axis 1
sns.pointplot(data=input_df, x="k", y="improvement", color=palette[1], capsize=0.3, ax=ax)#hue="horizon", ax=ax)
sns.despine(ax=ax)

ax.set_xlabel(r"\# of action changes, $k$")
ax.set_ylabel('Average c/f improvement')
ax.yaxis.set_major_formatter(mtick.PercentFormatter(decimals=0))

fig.tight_layout()
fig.savefig('figures/kappas_improvement_line.pdf', dpi=300)